In [1]:
import matplotlib.pyplot as plt
import numpy as np
from Nexullance_IT import Nexullance_IT
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from topologies.DDF import DDFtopo
import globals as gl
import copy
from joblib import Parallel, delayed
%matplotlib widget

# setup network, not yet using nexullance

In [2]:
config=(36, 5)
EPR=3
ddf_network = DDFtopo(config[0], config[1])
Cap_remote = 10 #GBps
Cap_local = 10 #GBps
M_EPs = gl.generate_uniform_traffic_pattern(config[0], EPR) # an numpy matrix filled with ones
scaling_factor = 1
M_EPs = scaling_factor * M_EPs
M_R = gl.convert_M_EPs_to_M_R(M_EPs, config[0], EPR)

In [3]:
ASP = ddf_network.calculate_all_shortest_paths()
ECMP_ASP = gl.ECMP(ASP)
remote_link_flows, local_link_flows = ddf_network.distribute_M_EPs_on_weighted_paths(ECMP_ASP, EPR, M_EPs)
max_remote_link_load = np.max(remote_link_flows)/Cap_remote
max_local_link_load = np.max(local_link_flows)/Cap_local
print("Max remote link load: ", max_remote_link_load)
print("Max local link load: ", max_local_link_load)
if max_remote_link_load > 1.0 or max_local_link_load > 1.0:
    print("network congestion may present")
    mu = max_local_link_load/max_remote_link_load
    if mu > 1.0:
        print(f"mu = {mu} > 1, max local link load is higher")
    else:
        print(f"mu = {mu} < 1, max remote link load is higher")
else:
    print("network is probably not congested")
print("network_total_throughput = ", gl.network_total_throughput(M_EPs, max_remote_link_load, max_local_link_load), "GBps")

Max remote link load:  20.775
Max local link load:  10.7
network congestion may present
mu = 0.5150421179302046 < 1, max remote link load is higher
network_total_throughput =  556.245487364621 GBps


In [4]:
# simply linearly scale everything:
traffic_scaling = 10.0/max(max_local_link_load, max_remote_link_load)
print(f"scaling traffic to:{traffic_scaling}x")
M_EPs = M_EPs*traffic_scaling
M_R = M_R*traffic_scaling

max_remote_link_load = traffic_scaling * max_remote_link_load
max_local_link_load = traffic_scaling * max_local_link_load
print("Max remote link load: ", max_remote_link_load)
print("Max local link load: ", max_local_link_load)
if max_remote_link_load > 1.0 or max_local_link_load > 1.0:
    print("network congestion may present")
    mu = max_local_link_load/max_remote_link_load
    if mu > 1.0:
        print(f"mu = {mu} > 1, max local link load is higher")
    else:
        print(f"mu = {mu} < 1, max remote link load is higher")
else:
    print("network is probably not congested")
print("network_total_throughput = ", gl.network_total_throughput(M_EPs, max_remote_link_load, max_local_link_load), "GBps")

scaling traffic to:0.48134777376654636x
Max remote link load:  10.0
Max local link load:  5.150421179302046
network congestion may present
mu = 0.5150421179302046 < 1, max remote link load is higher
network_total_throughput =  556.2454873646209 GBps


In [5]:
alpha_1 = 40.0
beta_1 = 0.4

alpha_2 = 1
beta_2 = 8

def weighted_method_1(s: int, d: int, edge_attributes: dict):
    return alpha_1 + edge_attributes['load']**beta_1

def weighted_method_2(s: int, d: int, edge_attributes: dict):
    return alpha_2 + edge_attributes['load']**beta_2

In [6]:
nexu_it = Nexullance_IT(ddf_network.nx_graph, M_R, Cap_remote)

In [7]:
num_method_1 = 2
num_method_2 = 10
results_method_1, results_method_2 = nexu_it.optimize(num_method_1, num_method_2, weighted_method_1, weighted_method_2)

In [8]:
results_method_1 = np.array([gl.network_total_throughput(M_EPs, results_method_1[i], max_local_link_load) for i in range(len(results_method_1))])
results_method_2 = np.array([gl.network_total_throughput(M_EPs, results_method_2[i][-1], max_local_link_load) for i in range(len(results_method_2))])
OPTIMUM = 775.0  # Nexullance_OPT [GBps]

In [9]:
results_method_1/OPTIMUM

array([0.71773611, 0.92043011])

In [10]:
results_method_2/OPTIMUM

array([0.92043011, 0.92043011, 0.93339391, 0.94672811, 0.9501214 ,
       0.95182718, 0.95225459, 0.95268238, 0.95289641, 0.953057  ])